In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
# 법령 XML 상세조회 (시행기준. target=eflaw)

#기본 url
url_base = " http://www.law.go.kr/DRF/lawService.do"

#조회 url 세부 설정
user_id = 'bmsong' #open API ID (bmsong@kau.ac.kr의 아이디 부분)
target = "eflaw" #법령 조회시
MST = "216209" #ID 또는 MST #전기안전관리법 MST 216209
#ID = "013718" 
            #ID 전기안전관리법 법령ID 013718 (설명에는 ID로 조회하면 현행법령이 조회된다는데, 
            #target을 eflaw로 하면 ID로 요청해도, MST로 요청해도 같은 것 같다.
Type = "XML" #출력 형태 : HTML 또는 XML

url_sub = "?" +\
        "OC=" + user_id +\
        "&target=" + target +\
        "&MST=" + MST +\
        "&type=" + Type

#최종 url
url = url_base + url_sub

print(url)

response = requests.get(url)

#(1) response.text 사용하는 방법
#soup = BeautifulSoup(response.text.encode('utf-8'), 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

#(2) response.content 사용하는 방법
soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)


# soup

 http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=eflaw&MST=216209&type=XML


In [3]:
lc_basic_df = pd.DataFrame() #DataFrame for Law Content 

data = soup.find('법령ID') #MST 정보
lc_basic_df['법령ID'] = [data.get_text()]

data = soup.find('법종구분') 
lc_basic_df['법종구분']= [data.get_text()]

data = soup.find('법령명_한글') 
lc_basic_df['법령명']= [data.get_text()]

data = soup.find('시행일자')
lc_basic_df['시행일자'] = [data.get_text()]

data = soup.find('공포일자') 
lc_basic_df['공포일자']= [data.get_text()]

data = soup.find('소관부처') 
lc_basic_df['소관부처']= [data.get_text()]

lc_basic_df

,법령ID,법종구분,법령명,시행일자,공포일자,소관부처
0,013718,법률,전기안전관리법,20210401,20200331,산업통상자원부


In [10]:
# data = soup.find_all('조문단위')
# df_info = pd.DataFrame()
# info = []
# for i in data:
#     detail = {}
#     detail["index"] = np.nan
#     detail["조문제목"] = np.nan
#     detail["조문내용"] = np.nan
#     detail["항문내용"] = np.nan
#     detail["호문내용"] = np.nan
#     if i.find("조문여부").get_text() == "조문":
#         조문제목 = i.find("조문내용").get_text().strip().split(")")[0]+")"
#         detail["조문제목"] = 조문제목
#         if i.find("조문내용"):
#             조문내용 = i.find("조문내용").get_text().strip()[len(조문제목):-1].strip()
#             detail["조문내용"] = 조문내용
        
#         df_info = df_info.append(detail,ignore_index = True)
        
#         if i.find("항"):
#             for j in i.find_all("항"):
#                 if j.find("항내용"):
#                     detail["항문내용"] = j.항내용.get_text().strip()
#                     df_info = df_info.append(detail,ignore_index = True)
#                 if j.find("호내용"):
#                     for k in j.find_all("호내용"):
#                         detail["호문내용"] = k.get_text().strip()
#                         df_info = df_info.append(detail,ignore_index = True)
#                 detail["호문내용"] = np.nan
# df_info = df_info[["조문제목","조문내용","항문내용","호문내용"]]
# df_info

,조문제목,조문내용,항문내용,호문내용
0,제1조(목적),이 법은 전기재해의 예방과 전기설비 안전관리에 필요한 사항을 규정함으로써 국민의 생...,NaN,NaN
1,제2조(정의),이 법에서 사용하는 용어의 뜻은 다음과 같다,NaN,NaN
2,제2조(정의),이 법에서 사용하는 용어의 뜻은 다음과 같다,NaN,"1. ""전기안전관리""란 국민의 생명과 재산을 보호하기 위하여 전기설비의 공사·유지·..."
3,제2조(정의),이 법에서 사용하는 용어의 뜻은 다음과 같다,NaN,"2. ""전기재해""란 전기화재, 감전사고 등으로 인하여 사람의 생명과 재산의 피해가 ..."
4,제2조(정의),이 법에서 사용하는 용어의 뜻은 다음과 같다,NaN,"3. ""전기사업자""란 「전기사업법」 제2조제2호에 따른 전기사업자를 말한다."
...,...,...,...,...
288,제52조(과태료),,② 다음 각 호의 어느 하나에 해당하는 자에게는 100만원 이하의 과태료를 부과한다.,9. 제25조제1항제2호를 위반하여 안전시공교육을 받지 아니한 사람
289,제52조(과태료),,② 다음 각 호의 어느 하나에 해당하는 자에게는 100만원 이하의 과태료를 부과한다.,10. 제25조제4항을 위반하여 전기안전교육을 받지 아니한 사람을 해임하지 아니한 자
290,제52조(과태료),,② 다음 각 호의 어느 하나에 해당하는 자에게는 100만원 이하의 과태료를 부과한다.,11. 제25조제5항을 위반하여 안전시공교육을 받지 아니한 사람의 시공관리책임자 지...
291,제52조(과태료),,② 다음 각 호의 어느 하나에 해당하는 자에게는 100만원 이하의 과태료를 부과한다.,12. 제26조제1항제4호에 따른 등록을 하지 아니하거나 같은 조 제2항에 따른 변...


In [4]:
data = soup.find_all('조문단위')
df_info = pd.DataFrame()
info = []
for i in data:
    info_str = ""
    if i.find("조문여부").get_text() == "조문":
        info_str += i.조문내용.get_text().strip()
        if i.항:
            hang_nums = i.find_all("항번호")
            ho_nums = i.find_all("호번호")
            for idx, p in enumerate(i.find_all("호내용")):
                info_str += p.get_text().split(ho_nums[idx].get_text())[1].strip()
            for hang_idx, k in enumerate(i.find_all("항내용")):
                info_str += k.get_text().strip().split(hang_nums[hang_idx].get_text())[1].strip()
                for ho_idx ,j in enumerate(i.find_all("호내용")):
                    info_str += j.get_text().strip().split(ho_nums[ho_idx].get_text())[1].strip()
        info.append(info_str)
df_info["조내용"] = info
df_info

,조내용
0,제1조(목적) 이 법은 전기재해의 예방과 전기설비 안전관리에 필요한 사항을 규정함으...
1,"제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.""전기안전관리""란 국민의..."
2,제3조(국가의 책무) 국가는 국민이 전기재해로부터 안전성을 확보할 수 있도록 전기안...
3,제4조(다른 법률과의 관계) 전기안전관리에 대하여 다른 법률에 특별한 규정이 있는 ...
4,제5조(전기안전관리 기본계획 수립 등)전기안전관리에 관한 중·장기 정책에 관한 사항...
5,제6조(전기안전자문기구의 설치)산업통상자원부장관은 전기안전관리 정책을 원활하게 수립...
6,"제7조(전기안전관리 관련 기술의 연구·개발)제30조에 따른 한국전기안전공사(이하 ""..."
7,제8조(자가용전기설비의 공사계획의 인가 또는 신고)자가용전기설비의 설치공사 또는 변...
8,제9조(사용전검사) 제8조에 따라 자가용전기설비의 설치공사 또는 변경공사를 한 자는...
9,제10조(자가용전기설비의 임시사용)산업통상자원부장관 또는 시·도지사는 제9조에 따른...


In [5]:
df_info.to_excel('전기안전관리법.xlsx',index = False)